In [5]:



import numpy as np
from scipy import stats, interpolate, fft, linalg
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

%matplotlib inline

In [ ]:



import numpy as np
from scipy import stats, interpolate
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation


rect = lambda x, center=0, width=1, hight=1.: hight*(np.abs(x-center)<width/2) 
exp = lambda x, center=0, lam=1, amp=1.: amp*np.exp(-lam*(x-center) )*(x-center>=0)
gaussian = lambda x, center=0, sigma=1, amp=1: amp*stats.norm.pdf( (x-center)/sigma )



def barker_code(t, center=0, expansion=13):
    t = t-center
    bc = np.array([+1, +1, +1, +1, +1, -1, -1, +1, +1, -1, +1, -1, +1])
    t_n = np.arange(
        int(np.min(t)//1-2-len(bc)-expansion), 
        int(np.max(t)//1+2+len(bc)+expansion)
    )
    code = np.zeros_like(t_n)
    pos_zero = int(*np.where(t_n==0))
    code[pos_zero:pos_zero+13] = bc
    func_sig = interpolate.interp1d(t_n, code, kind='previous')
    sig = func_sig(t)
    return sig


def sinc(t, center=0):
    y = np.sinc(t-center)
    return y

frame_to_frequency = lambda n, total=200, start=0, stop=5: n/total * (stop-start) + start


def build_signal(x, sig, center=0, *args, **kwargs):
    if 'center' not in kwargs.keys():
        kwargs['center'] = center
    elif 'center' in kwargs.keys():
        kwargs['center'] += center
    y = sig(x, **kwargs)
    return y


def plot_sig1_sig2_conv(x, sig1_y, sig2_y, delay, conv12, plt_lims, fig, gs, ax):
    ax1, ax2 = ax
    sig_product = sig1_y * sig2_y
    conv = np.dot(sig1_y, sig2_y)*(x[1]-x[0])
    #x_prime.append(delay), y_prime.append(conv)
    #
    ax1.cla()
    ax1.set_title('Correlation')
    ax1.set_xlim(min(x), max(x))
    ax1.set_ylim(plt_lims['y1'])
    ax1.plot(x, sig_product, 'k', label='f(t)g(t+tau)')
    ax1.plot(x, sig1_y, 'b', label='f(t)')
    ax1.plot(x, sig2_y, 'r', label='g(t+tau)')
    ax1.set_xlabel('Time')
    ax1.set_ylabel('Value')
    ax1.fill_between(x, sig_product, color='b', alpha=0.2)
    ax1.legend(loc='upper right')
    ax1.grid(True)
    #
    ax2.cla()
    ax2.set_title('')
    ax2.set_xlim((min(x), max(x)))
    ax2.set_ylim(plt_lims['y2'])
    ax2.set_xlabel('(f*g)(t)')
    ax2.plot(delay, conv12, 'k', label='')
    ax2.grid(True)




conv_func = {
    'rect': {
        'sig': rect, 'sig_kwargs': {'center': 0},
        'x_params': (-5, 5, 1001), 'freq':(0, 10)'expansion': 0,
        'plt_lims': {
            'y1': (-0.1, 1.1), 'y2': (-0.1, 1.1)
        },
    },
    'sinc': {
        'sig': sinc, 'sig_kwargs':{},
        'x_params':(-10, 10, 10001), 'expansion': 10,
        'plt_lims': {
            'y1': (-0.4, 1.1), 'y2': (-0.4, 1.2),
        },
    },
    'barker' : {
        'sig': barker_code, 'sig_kwargs':{'center': 0},
        'x_params':(-30, 30, 6001), 'expansion': 13,
        'plt_lims': {
            'y1': (-1.2, 1.2), 'y2': (-1, 14)
        },
    },
    'rect_fat_rect': {
        'sig1': rect, 'sig1_kwargs': {'center': 0, 'width':2},
        'x_params': (-3, 3, 1001), 'expansion': 0.5,
        'plt_lims': {
            'y1': (-0.1, 1.1), 'y2': (-0.1, 1.1)
        },
    },
    'exp_exp': {
        'sig1':  exp, 'sig1_kwargs': {'center': 0},
        'sig2':  exp, 'sig2_kwargs': {},
        'x_params': (-5, 5, 1001), 'expansion': 0,
        'plt_lims': {
            'y1': (-0.1, 1.1), 'y2': (-0.1, 1.1)
        },
    },
    'rect_gaussian': {
        'sig1': rect, 'sig1_kwargs': {'center': 0},
        'sig2': gaussian, 'sig2_kwargs': {},
        'x_params': (-5, 5, 1001), 'expansion': 3,
        'plt_lims': {
            'y1': (-0.1, 1.1), 'y2': (-0.1, 1.1)
        },
    },
    'gaussian_gaussian': {
        'sig1': gaussian, 'sig1_kwargs': {'center': 0},
        'sig2': gaussian, 'sig2_kwargs': {},
        'x_params': (-5, 5, 1001), 'expansion': 3,
        'plt_lims': {
            'y1': (-0.1, 1.1), 'y2': (-0.1, 1.1)
        },
    },
    
}
    
total_frames = 400


for name, params in conv_func.items():
    file_name = 'Correlation_'+name+'.gif'
    sig, sig1_kwargs = params['sig'], params['sig_kwargs']
    x_params, freq = params['x_params'], params['freq']
    plt_lims = params['plt_lims']
    
    x = np.linspace(*x_params)
    sig1_y = build_signal(x, sig1, **sig1_kwargs)
    
    fig = plt.figure(constrained_layout=True)
    fig.set_size_inches(8, 4)
    gs = fig.add_gridspec(4, 8)
    ax1 = fig.add_subplot(gs[0:2, :])
    ax2 = fig.add_subplot(gs[2:4, :])
    ax = (ax1, ax2)
    
    delay_array, conv_array = [], []
    
    def plot_conv_animate(frame):
        f = frame_to_delay(
            frame, total=total_frames, start=freq[0], 
            stop=freq[1])
        
        
        
        if 'center' in sig2_kwargs.keys():
            tau -= sig2_kwargs['center']
            del sig2_kwargs_2nd['center']
        
        sig2_y = build_signal(x, sig2, center=tau, **sig2_kwargs_2nd)
        conv = np.dot(sig1_y, sig2_y)*(x[1]-x[0])
        
        delay_array.append(t)
        conv_array.append(conv)
        
        plot_sig1_sig2_conv(x, sig1_y, sig2_y, delay_array, conv_array, 
                            plt_lims, fig, gs, ax)
    
    anim = FuncAnimation(fig, plot_conv_animate, frames=range(total_frames), interval=40, blit=False)
    
    print(file_name)
    anim.save(file_name)
    print('Done')
    break
    
    
print('Done! '*4)